## Decoupling with SQS and SNS

The deployed architecture decouples the Lambda functions from each other by using Amazon SQS and Amazon SNS. The Amazon API Gateway integrates with a Lambda function, which accepts an order that is posted by the client. The function extracts the body from the event and publishes it as a message to an Amazon SNS topic.

The SNS topic sends the published message to a couple of Amazon SQS queues, which are subscribers. The Lambda functions associated with each queue is invoked when the message arrives in the queue.

The invoice Lambda function writes the message to an Amazon S3 bucket, this is saved as an object. 

The order Lambda function writes messages to an Amazon DynamoDB table, this is saved as an item. The Amazon DynamoDB table is partitioned on an accountid attribute and also includes a sort key on the vendorid attribute, together they form the primary key.

Additional Lambda functions are deployed to get the order item from Amazon DynamoDB and the object from the Amazon S3 bucket.

Observability has also been built into the architecture with X-Ray instrumentation and CloudWatch Lambda Insights Extensions. 


![architecture](../images/architecture_1.png "Architecture")

### Jupyter Notebook Scripts

1.	The first step in the demonstration is to enable X-Ray tracing on the API


2.	The first script posts new orders to the API Gateway. The order is written to an Amazon DynamoDB table and invoice is written to an Amazon S3 bucket.


3.	The second script creates a json formatter which is used to render the json output in a readable format.


4.	The third script is used to query items in the Amazon DynamoDB table.


5.	The fourth script is used to generate a pre-signed URL used to get the object from the S3 bucket.


6.	Open X-Ray to view the end to end trace. The trace will highlight any latency or errors.


7.	Open Lambda insights and view the performance monitoring metrics captured by the CloudWatch Insights Extension. 

### Task 1 - Enable Tracing
Navigate to the sqs_sns_lambda_dynamodb API and check the box to Enable X-Ray Tracing on the prod stage. Make sure you Save Changes.

![feature](../images/feature_1.png " feature ")


### Task 2 – POST New Order

**Set variables**
- Make sure you set **gwid** to your gateway id using `gwid = '...'`
- Make sure you set **region** to your preferred region using `region = '...'`

The script will POST new items to the DynamoDB table, this may take up to 30 seconds to complete.

In [ ]:
import boto3, requests, datetime
from random import randrange

#Set gateway id
gwid = '...'

#Set your AWS region, e.g. ap-southeast-2
region = '...' 

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":"3.5"},{"size":"Medium","price":"4.0"},{"size":"Large","price":"4.5"},{"size":"x-Large","price":"5.0"}]

url = (f'https://{gwid}.execute-api.{region}.amazonaws.com/prod/order')

for i in range(10000): #Increase the counter to generate more cloudwatch metrics data
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    coffeesize = randrange(4)
    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    eventtype="new_order"
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype_list[randrange(4)],
                'coffeesize': coffeesize_list[coffeesize]["size"],
                'unitprice': coffeesize_list[coffeesize]["price"],
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    })

print(response)

### Task 3 – JSON Formatter

The following script create a class display nicely formatted json data

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

### Task 4 – Get Order

Run the following script to return a item from Amazon DynamoDB using the primary key (accountid & vendorid)


In [ ]:
url = (f'https://{gwid}.execute-api.{region}.amazonaws.com/prod/order')

response_get = requests.get(url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

### Task 5 – Get Invoice

The following script calls an API endpoint which return a pre-signed URL allowing you to download the invoice from the Amazon S3 without making the bucket public.

In [ ]:
url = (f'https://{gwid}.execute-api.{region}.amazonaws.com/prod/invoice')

response_get = requests.get(url, params={'accountid':'a0'})

print(response_get.json())

### Task 6 – X-Ray

Open the X-Ray console. The following diagram should resemble the service map. 
![feature](../images/feature_2.png " feature ")

The sqs_sns_lambda_dynamodb_api Lambda function add **Annotations** relating to order details and **Metadata** related to the execution context.

![feature](../images/feature_3.png " feature ")


### Task 7 – Lambda Insights
Navigate to CloudWatch Lambda Insights and set the **Performance monitoring** to **Multi-function**. You can see from the dashboard the three functions involved in the application vary in duration and resources used. 

The **sqs_sns_dynamodb_put_object** function, which writes to S3, has not been allocated the same resources as the two other functions, therefore it is slower. However, this does not impact the performance of the other functions because they are communicating asynchronously.

Function name: ```sqs_sns_dynamodb_api```
Memory: ```512```

Function name: ```sqs_sns_dynamodb_put_item```
Memory: ```512```
SQS Batch size: ```10```

Function name: ```sqs_sns_dynamodb_put_object```
Memory: ```128```
SQS Batch size: ```5```



**Lambda Insight Errors**

If you set the loop counter high enough, 10000+, you will start to see the sqs_sns_dynamodb_put_item function produce errors. The following diagram is take from Lambda Insights.

![Lambda_insights](../images/Lambda_insights.png "Lambda_insights")

**DynamoDB Capacity**

The errors are due to the burst thoughput of the DynamoDB table being depleted and with only 5 write capacity units the DynamoDB table is under provisioned.

![table_metrics](../images/table_metrics.png "table_metrics")
